# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
from scipy import stats
import math


## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from hidden import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 55605867, 'calculationPrice': 'close', 'change': -0.59, 'changePercent': -0.00347, 'close': 169.3, 'closeSource': 'official', 'closeTime': 1714161600010, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 169.31, 'delayedPriceTime': 1714161590054, 'extendedChange': 0.36, 'extendedChangePercent': 0.00213, 'extendedPrice': 169.66, 'extendedPriceTime': 1714175992869, 'high': 171.34, 'highSource': '15 minute delayed price', 'highTime': 1714161599997, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 169.19, 'iexCloseTime': 1714161599303, 'iexLastUpdated': 1714161599303, 'iexMarketPercent': 0.011971135247292977, 'iexOpen': 169.86, 'iexOpenTime': 1714138200024, 'iexRealtimePrice': 169.19, 'iexRealtimeSize': 100, 'iexVolume': 536766, 'lastTradeTime': 1714161599997, 'latestPrice': 169.3, 'latestSource': 'Close', 'latestTime': 'April 26, 2024', 'latestUpdate': 1714161600010, 'latestVolume': 44838354, 'low': 169.18, 'lowSource': '15 

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']

pe_ratio

26.37

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        try:
            final_dataframe = final_dataframe._append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],index = my_columns
            ),ignore_index=True
            )
        except KeyError:
            print(f"Symbol '{symbol}' not found in data")
            
final_dataframe

Symbol 'ABC' not found in data
Symbol 'ANTM' not found in data
Symbol 'DISCA' not found in data
Symbol 'FBHS' not found in data
Symbol 'HFC' not found in data
Symbol 'NLOK' not found in data
Symbol 'RE' not found in data
Symbol 'VIAC' not found in data
Symbol 'WLTW' not found in data


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,137.74,32.87,N/A
1,AAL,13.88,21.17,N/A
2,AAP,75.14,150.64,N/A
3,AAPL,169.30,26.37,N/A
4,ABBV,159.62,58.68,N/A
...,...,...,...,...
491,YUM,141.77,25.32,N/A
492,ZBH,119.35,24.46,N/A
493,ZBRA,297.21,51.96,N/A
494,ZION,41.76,9.58,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio',inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0] 
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index',axis=1,inplace=True)


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [12]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row,'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BRK.B,402.100,0.01,49
1,INFO,0.201,0.07,99502
2,FRC,3.510,0.43,5698
3,DISH,5.770,2.90,3466
4,APA,32.490,3.52,615
5,FMC,58.460,5.56,342
6,GM,45.840,5.65,436
7,EMR,109.890,5.72,182
8,DAL,49.920,6.42,400
9,SYF,44.670,6.44,447


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [40]:
symbol = 'AAPL'
batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol},fb&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
#print(data['AAPL']['advanced-stats'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']
# Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']
# Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit
print(ev_to_gross_profit)



15.254420446130903


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [53]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]
rv_dataframe = pd.DataFrame(columns = rv_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            gross_profit = data[symbol]['advanced-stats']['grossProfit']
            try:
                ev_to_ebitda = enterprise_value/ebitda
            except TypeError:
                ev_to_ebitda = np.NaN

            try:
                ev_to_gross_profit = enterprise_value/gross_profit
            except TypeError:
                ev_to_gross_profit = np.NaN
            rv_dataframe = rv_dataframe._append(
                pd.Series([
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
            ],
            index = rv_columns),
                ignore_index = True
            )
        except KeyError:
            print(f"Symbol '{symbol}' not found in data")

Symbol 'ABC' not found in data
Symbol 'ANTM' not found in data
Symbol 'DISCA' not found in data
Symbol 'FBHS' not found in data
Symbol 'HFC' not found in data
Symbol 'NLOK' not found in data
Symbol 'RE' not found in data
Symbol 'VIAC' not found in data
Symbol 'WLTW' not found in data


In [54]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.74,N/A,32.87,N/A,6.52,N/A,5.9900,N/A,26.157837,N/A,12.091758,N/A,N/A
1,AAL,13.88,N/A,21.17,N/A,-1.66,N/A,0.1713,N/A,5.969471,N/A,0.623230,N/A,N/A
2,AAP,75.14,N/A,150.64,N/A,1.78,N/A,0.3968,N/A,13.691828,N/A,1.273780,N/A,N/A
3,AAPL,169.30,N/A,26.37,N/A,35.28,N/A,6.7800,N/A,20.361777,N/A,15.254420,N/A,N/A
4,ABBV,159.62,N/A,58.68,N/A,27.28,N/A,5.2000,N/A,14.926982,N/A,9.710045,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,141.77,N/A,25.32,N/A,-5.08,N/A,5.6400,N/A,20.599045,N/A,9.541919,N/A,N/A
492,ZBH,119.35,N/A,24.46,N/A,1.96,N/A,3.3200,N/A,14.841973,N/A,5.625551,N/A,N/A
493,ZBRA,297.21,N/A,51.96,N/A,5.03,N/A,3.3300,N/A,25.194282,N/A,8.176571,N/A,N/A
494,ZION,41.76,N/A,9.58,N/A,1.17,N/A,1.3300,N/A,6.007539,N/A,1.372158,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [58]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
5,ABMD,381.020,N/A,65.69,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
27,ALXN,182.500,N/A,59.45,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
38,AON,285.030,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
46,ATVI,94.420,N/A,34.59,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
86,CERN,94.920,N/A,47.94,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
116,CTL,11.000,N/A,9.73,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
119,CTXS,103.900,N/A,39.96,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
133,DISCK,24.420,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
134,DISH,5.770,N/A,2.90,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
162,ETFC,49.260,N/A,14.27,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [62]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(),inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [63]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [64]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
for metric in metrics.values():
    print(rv_dataframe[metric])
rv_dataframe

0      0.735887
1      0.493952
2      0.989919
3      0.610887
4      0.923387
         ...   
491    0.594758
492      0.5625
493    0.897177
494    0.157258
495    0.702621
Name: PE Percentile, Length: 496, dtype: object
0      0.772177
1        0.0625
2      0.287298
3      0.975806
4      0.967742
         ...   
491    0.056452
492    0.327621
493    0.699597
494    0.138105
495    0.921371
Name: PB Percentile, Length: 496, dtype: object
0      0.822581
1      0.010081
2      0.046371
3      0.850806
4      0.774194
         ...   
491    0.802419
492    0.594758
493    0.596774
494    0.245968
495    0.907258
Name: PS Percentile, Length: 496, dtype: object
0      0.887097
1      0.074597
2       0.47379
3      0.782258
4      0.568548
         ...   
491    0.790323
492    0.566532
493    0.877016
494    0.078629
495    0.846774
Name: EV/EBITDA Percentile, Length: 496, dtype: object
0      0.820565
1      0.018145
2        0.0625
3      0.909274
4      0.703629
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.74,N/A,32.87,0.735887,6.52,0.772177,5.9900,0.822581,26.157837,0.887097,12.091758,0.820565,N/A
1,AAL,13.88,N/A,21.17,0.493952,-1.66,0.0625,0.1713,0.010081,5.969471,0.074597,0.623230,0.018145,N/A
2,AAP,75.14,N/A,150.64,0.989919,1.78,0.287298,0.3968,0.046371,13.691828,0.47379,1.273780,0.0625,N/A
3,AAPL,169.30,N/A,26.37,0.610887,35.28,0.975806,6.7800,0.850806,20.361777,0.782258,15.254420,0.909274,N/A
4,ABBV,159.62,N/A,58.68,0.923387,27.28,0.967742,5.2000,0.774194,14.926982,0.568548,9.710045,0.703629,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,141.77,N/A,25.32,0.594758,-5.08,0.056452,5.6400,0.802419,20.599045,0.790323,9.541919,0.691532,N/A
492,ZBH,119.35,N/A,24.46,0.5625,1.96,0.327621,3.3200,0.594758,14.841973,0.566532,5.625551,0.427419,N/A
493,ZBRA,297.21,N/A,51.96,0.897177,5.03,0.699597,3.3300,0.596774,25.194282,0.877016,8.176571,0.618952,N/A
494,ZION,41.76,N/A,9.58,0.157258,1.17,0.138105,1.3300,0.245968,6.007539,0.078629,1.372158,0.086694,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [65]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.74,N/A,32.87,0.735887,6.52,0.772177,5.9900,0.822581,26.157837,0.887097,12.091758,0.820565,0.807661
1,AAL,13.88,N/A,21.17,0.493952,-1.66,0.0625,0.1713,0.010081,5.969471,0.074597,0.623230,0.018145,0.131855
2,AAP,75.14,N/A,150.64,0.989919,1.78,0.287298,0.3968,0.046371,13.691828,0.47379,1.273780,0.0625,0.371976
3,AAPL,169.30,N/A,26.37,0.610887,35.28,0.975806,6.7800,0.850806,20.361777,0.782258,15.254420,0.909274,0.825806
4,ABBV,159.62,N/A,58.68,0.923387,27.28,0.967742,5.2000,0.774194,14.926982,0.568548,9.710045,0.703629,0.7875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,141.77,N/A,25.32,0.594758,-5.08,0.056452,5.6400,0.802419,20.599045,0.790323,9.541919,0.691532,0.587097
492,ZBH,119.35,N/A,24.46,0.5625,1.96,0.327621,3.3200,0.594758,14.841973,0.566532,5.625551,0.427419,0.495766
493,ZBRA,297.21,N/A,51.96,0.897177,5.03,0.699597,3.3300,0.596774,25.194282,0.877016,8.176571,0.618952,0.737903
494,ZION,41.76,N/A,9.58,0.157258,1.17,0.138105,1.3300,0.245968,6.007539,0.078629,1.372158,0.086694,0.141331


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [67]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

/tmp/ipykernel_52004/3864872901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values(by = 'RV Score', inplace = True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,N/A,0.43,0.08871,0.0456,0.064516,0.0934,0.002016,0.270279,0.016129,0.094878,0.004032,0.035081
1,LNC,27.81,N/A,-6.31,0.070565,0.8004,0.080645,0.2963,0.030242,-5.156128,0.014113,0.296325,0.006048,0.040323
2,KSS,24.53,N/A,8.58,0.137097,0.6988,0.072581,0.1557,0.008065,4.795732,0.034274,1.007530,0.046371,0.059677
3,GM,45.84,N/A,5.65,0.096774,0.7853,0.076613,0.2991,0.032258,1.902156,0.018145,1.357213,0.082661,0.06129
4,UNM,50.75,N/A,7.81,0.120968,1.0100,0.102823,0.7864,0.118952,4.591120,0.032258,0.786383,0.02621,0.080242
5,DXC,20.08,N/A,-8.85,0.056452,1.1800,0.142137,0.2647,0.020161,3.611319,0.024194,2.075761,0.165323,0.081653
6,UA,6.55,N/A,7.36,0.108871,1.2300,0.153226,0.4633,0.05746,5.593101,0.061492,0.875193,0.037298,0.083669
7,UAA,6.80,N/A,7.64,0.117944,1.2300,0.153226,0.4633,0.05746,5.593101,0.061492,0.875193,0.037298,0.085484
8,SYF,44.67,N/A,6.44,0.102823,1.2800,0.165323,0.7399,0.100806,4.183059,0.028226,1.014304,0.048387,0.089113
9,CVS,67.18,N/A,10.38,0.179435,1.1100,0.116935,0.2367,0.012097,7.109626,0.131048,0.958459,0.044355,0.096774


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [69]:
portfolio_input()

Enter the value of your portfolio:1000000


In [70]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,5698,0.43,0.08871,0.0456,0.064516,0.0934,0.002016,0.270279,0.016129,0.094878,0.004032,0.035081
1,LNC,27.81,719,-6.31,0.070565,0.8004,0.080645,0.2963,0.030242,-5.156128,0.014113,0.296325,0.006048,0.040323
2,KSS,24.53,815,8.58,0.137097,0.6988,0.072581,0.1557,0.008065,4.795732,0.034274,1.007530,0.046371,0.059677
3,GM,45.84,436,5.65,0.096774,0.7853,0.076613,0.2991,0.032258,1.902156,0.018145,1.357213,0.082661,0.06129
4,UNM,50.75,394,7.81,0.120968,1.0100,0.102823,0.7864,0.118952,4.591120,0.032258,0.786383,0.02621,0.080242
5,DXC,20.08,996,-8.85,0.056452,1.1800,0.142137,0.2647,0.020161,3.611319,0.024194,2.075761,0.165323,0.081653
6,UA,6.55,3053,7.36,0.108871,1.2300,0.153226,0.4633,0.05746,5.593101,0.061492,0.875193,0.037298,0.083669
7,UAA,6.80,2941,7.64,0.117944,1.2300,0.153226,0.4633,0.05746,5.593101,0.061492,0.875193,0.037298,0.085484
8,SYF,44.67,447,6.44,0.102823,1.2800,0.165323,0.7399,0.100806,4.183059,0.028226,1.014304,0.048387,0.089113
9,CVS,67.18,297,10.38,0.179435,1.1100,0.116935,0.2367,0.012097,7.109626,0.131048,0.958459,0.044355,0.096774


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [71]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [72]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [73]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [74]:
writer.close()